<a href="https://colab.research.google.com/github/stinajoss/Bibliometri_GU/blob/main/gu_openalex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests

mailto = "lars.kullman@ub.gu.se"  # @param {type:"string"}

ror_id     = "01tm6cn81"  # @param {type:"string"}
start_year = 2018  # @param {type:"slider", min:2015, max:2023, step:1}
end_year   = 2023  # @param {type:"slider", min:2015, max:2023, step:1}

filter = f"authorships.institutions.ror:{ror_id},publication_year:{start_year}-{end_year}"
api_url = f"https://api.openalex.org/works?filter={filter}&per-page=200&mailto={mailto}"

cursor = "*"  # Cursor for the first request
works = []  # List to accumulate all results

while cursor:
    # Construct the request URL with the current cursor
    request_url = api_url + f"&cursor={cursor}"

    # Get results from the API
    page = requests.get(request_url).json()
    results = page.get('results')
    meta = page.get('meta')

    # Accumulate results
    works += results

    # Get the next cursor from the current results
    cursor = meta.get('next_cursor')

    # Print progress
    total = int(meta.get("count"))
    print(f"\r{len(works)}/{total}: {request_url}", end="")

# Do something with all results
print(f"\nTotal works: {len(works)}")

35182/35182: https://api.openalex.org/works?filter=authorships.institutions.ror:01tm6cn81,publication_year:2018-2023&per-page=200&mailto=lars.kullman@ub.gu.se&cursor=Ils2NS4wLCAwLCAnaHR0cHM6Ly9vcGVuYWxleC5vcmcvVzQzOTEwNzM3NjAnXSI=
Total works: 35182


In [ ]:
import pandas as pd

# Create a list of dictionaries with the extracted fields
data = []
for work in works:
    id = work.get('id').replace("https://openalex.org/", "")
    publication_year = work.get('publication_year')
    doi = work.get('doi')
    if doi:
        doi = doi.replace("https://doi.org/", "")
    else:
        doi = "N/A"  # Set a default value if 'doi' is None

    pubtype = work['type']

    # Check if 'title' exists and is not None
    if work.get('title'):
        title = work['title'][:99]
    else:
        title = "N/A"  # Set a default value if 'title' is None

    # Check if 'primary_location' exists and is not None
    if work.get('primary_location') and work['primary_location'].get('source'):
        source = work['primary_location']['source']['display_name']
    else:
        source = "N/A"  # Set a default value if 'primary_location' or 'source' is None

    is_oa = work['open_access']['is_oa']
    oa_status = work['open_access']['oa_status']
    oa_url = work['open_access']['oa_url']

    # Append the extracted fields to the list of dictionaries
    data.append({
        'id': id,
        'publication_year': publication_year,
        'doi': doi,
        'Publication_type': pubtype,
        'title': title,
        'source': source,
        'is_oa': is_oa,
        'oa_status': oa_status,
        'oa_url': oa_url
    })

# Create a Pandas DataFrame from the list of dictionaries
df = pd.DataFrame(data)

# Specify the output file name (change as needed)
output_file = 'gu_openalex_2018-2023.xlsx'

# Export the DataFrame to an Excel file
df.to_excel(output_file, index=False)

print(f"DataFrame exported to {output_file}")


DataFrame exported to gu_openalex_2018-2023.csv
